In [1]:
#Loading Libraries
import numpy as np 
import pandas as pd
import numpy as np
import os
import imutils
import dlib 
import cv2 
import imageio
from imutils import face_utils

In [2]:
def rect_to_bb(rect):
    # take a bounding predicted by dlib and convert it
    # to the format (x, y, w, h) as we would normally do
    # with OpenCV
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y

    # return a tuple of (x, y, w, h)
    return (x, y, w, h)

def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((68, 2), dtype=dtype)

    # loop over the 68 facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, 68):
    	coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords

In [3]:
def crop_and_save_image(img, img_path, write_img_path, img_name):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('D:/Lipreading/MIRACL-VC1_all_in_one/shape_predictor_68_face_landmarks.dat')
    # load the input image, resize it, and convert it to grayscale

    image = cv2.imread(img_path)
    image = imutils.resize(image, width=500)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # detect faces in the grayscale image
    rects = detector(gray, 1)
    if len(rects) > 1:
        print( "ERROR: more than one face detected")
        return
    if len(rects) < 1:
        print( "ERROR: no faces detected")
        return

    for (i, rect) in enumerate(rects):
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        name, i, j = 'mouth', 48, 68
        clone = gray.copy()

        (x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]]))
        
        # Calculate aspect ratio of the ROI
        aspect_ratio = w / h

        # Resize ROI to fit within a 40x40 box without distorting the aspect ratio
        if aspect_ratio > 1:  # Width > Height
            new_w = 40
            new_h = int(40 / aspect_ratio)
        else:  # Height >= Width
            new_w = int(40 * aspect_ratio)
            new_h = 40

        roi = gray[y:y+h, x:x+w]
        roi = cv2.resize(roi, (new_w, new_h), interpolation=cv2.INTER_AREA)

        # Pad the resized ROI to make it 40x40
        pad_top = (40 - new_h) // 2
        pad_bottom = 40 - new_h - pad_top
        pad_left = (40 - new_w) // 2
        pad_right = 40 - new_w - pad_left
        roi = cv2.copyMakeBorder(roi, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=0)

        print('cropped/' + write_img_path)
        imageio.imwrite('cropped/' + write_img_path, roi)


In [4]:
os.listdir('D:/Lipreading/MIRACL-VC1_all_in_one')
predictor = dlib.shape_predictor('D:/Lipreading/MIRACL-VC1_all_in_one/shape_predictor_68_face_landmarks.dat')

In [5]:

people = ['F01','F02','F04','F05','F06','F07','F08','F09', 'F10','F11','M01','M02','M04','M07','M08']
data_types = ['words']
folder_enum = ['01','02','03','04','05','06','07','08','09','10']
instances = ['01','02','03','04','05','06','07','08','09','10']

words = ['Begin', 'Choose', 'Connection', 'Navigation', 'Next', 'Previous', 'Start', 'Stop', 'Hello', 'Web']          
words_di = {i:words[i] for i in range(len(words))}

In [6]:
#crearting new folder for cropped images
#if not os.path.exists('cropped'):
    #os.mkdir('cropped')

In [7]:
import shutil

def crop_one_person():      
    
#    people = ['F01','F02','F04','F05','F06','F07','F08','F09', 'F10','F11','M01','M02','M04','M07','M08']
#    data_types = ['words']
#    folder_enum = ['01','02']
#    instances = ['01','02']

    i = 1
    for person_ID in people:
        if not os.path.exists('cropped/' + person_ID ):
            os.mkdir('cropped/' + person_ID + '/')

        for data_type in data_types:
            if not os.path.exists('cropped/' + person_ID + '/' + data_type):
                os.mkdir('cropped/' + person_ID + '/' + data_type)

            for phrase_ID in folder_enum:
                if not os.path.exists('cropped/' + person_ID + '/' + data_type + '/' + phrase_ID):
                    # F01/phrases/01
                    os.mkdir('cropped/' + person_ID + '/' + data_type + '/' + phrase_ID)

                for instance_ID in instances:
                    # F01/phrases/01/01
                    directory = 'D:/Lipreading/MIRACL-VC1_all_in_one/' + person_ID + '/' + data_type + '/' + phrase_ID + '/' + instance_ID + '/'
                    dir_temp = person_ID + '/' + data_type + '/' + phrase_ID + '/' + instance_ID + '/'
                    print(directory)
                    filelist = os.listdir(directory)
                    if not os.path.exists('cropped/' + person_ID + '/' + data_type + '/' + phrase_ID + '/' + instance_ID):
                        os.mkdir('cropped/' + person_ID + '/' + data_type + '/' + phrase_ID + '/' + instance_ID)

                        for img_name in filelist:
                            if img_name.startswith('color'):
                                image = imageio.imread(directory + '' + img_name)
                                crop_and_save_image(image, directory + '' + img_name,
                                                    dir_temp + '' + img_name, img_name)

    print(f'Iteration : {i}')
    i += 1
  #  shutil.rmtree('cropped')

In [8]:
import time
os.mkdir('cropped')
times = 0
for _ in range(7):
    t1 = time.time()
    crop_one_person()
    t2 = time.time()
    times += (t2 - t1)

print("Average time over 7 iterations : ", times/7)

D:/Lipreading/MIRACL-VC1_all_in_one/F01/words/01/01/


C:\Users\satvi\AppData\Local\Temp\ipykernel_20316\1155548971.py:35: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(directory + '' + img_name)


cropped/F01/words/01/01/color_001.jpg
cropped/F01/words/01/01/color_002.jpg
cropped/F01/words/01/01/color_003.jpg
cropped/F01/words/01/01/color_004.jpg
cropped/F01/words/01/01/color_005.jpg
cropped/F01/words/01/01/color_006.jpg
cropped/F01/words/01/01/color_007.jpg
cropped/F01/words/01/01/color_008.jpg
cropped/F01/words/01/01/color_009.jpg
cropped/F01/words/01/01/color_010.jpg
D:/Lipreading/MIRACL-VC1_all_in_one/F01/words/01/02/
cropped/F01/words/01/02/color_001.jpg
cropped/F01/words/01/02/color_002.jpg
cropped/F01/words/01/02/color_003.jpg
cropped/F01/words/01/02/color_004.jpg
cropped/F01/words/01/02/color_005.jpg
cropped/F01/words/01/02/color_006.jpg
cropped/F01/words/01/02/color_007.jpg
D:/Lipreading/MIRACL-VC1_all_in_one/F01/words/01/03/
cropped/F01/words/01/03/color_001.jpg
cropped/F01/words/01/03/color_002.jpg
cropped/F01/words/01/03/color_003.jpg
cropped/F01/words/01/03/color_004.jpg
cropped/F01/words/01/03/color_005.jpg
cropped/F01/words/01/03/color_006.jpg
cropped/F01/words/01

In [8]:
import numpy as np
import cv2

def batch_normalization(orig_seq):
    orig_len = len(orig_seq)
    new_seq = []
    for i in range(36):
        new_index = int((i * orig_len) / 36)
        new_seq.append(orig_seq[new_index])
    return new_seq

def concatenate_images(directory):
    images = []
    filelist = sorted(os.listdir(directory))
    for img_name in filelist:
        if img_name.endswith('.jpg'):
            img_path = os.path.join(directory, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (40, 40))  # Resize to 40x40 if not already resized
            images.append(img)

    # Apply batch normalization to the images
    normalized_images = batch_normalization(images)

    # Create a blank canvas for the concatenated image
    concatenated_image = np.zeros((240, 240), dtype=np.uint8)

    # Arrange the normalized images into a 4x4 grid
    for i in range(6):
        for j in range(6):
            idx = i * 6 + j
            img = normalized_images[idx]
            row_start = i * 40
            row_end = row_start + 40
            col_start = j * 40
            col_end = col_start + 40
            concatenated_image[row_start:row_end, col_start:col_end] = img

    return concatenated_image

# Example usage:
#person_ID = 'F01'
#data_type = 'words'
#phrase_ID = '01'
#instance_ID = '01'
#directory = f'cropped/{person_ID}/{data_type}/{phrase_ID}/{instance_ID}'
#concatenated_image = concatenate_images(directory)

# Display the concatenated image
#cv2.imshow('Concatenated Image', concatenated_image)
#cv2.waitKey(0)
#cv2.destroyAllWindows()


In [9]:
person_ID = ['F01','F02','F04','F05','F06','F07','F08','F09', 'F10','F11','M01','M02','M04','M07','M08']
def save_concatenated_images(root_dir, save_dir):
    for person_ID in os.listdir(root_dir):
        person_dir = os.path.join(root_dir, person_ID)
        if os.path.isdir(person_dir):
            for data_type in os.listdir(person_dir):
                data_dir = os.path.join(person_dir, data_type)
                if os.path.isdir(data_dir):
                    for phrase_ID in os.listdir(data_dir):
                        phrase_dir = os.path.join(data_dir, phrase_ID)
                        if os.path.isdir(phrase_dir):
                            for instance_ID in os.listdir(phrase_dir):
                                instance_dir = os.path.join(phrase_dir, instance_ID)
                                if os.path.isdir(instance_dir):
                                    concatenated_image = concatenate_images(instance_dir)
                                    save_path = os.path.join(save_dir, person_ID, data_type, phrase_ID, instance_ID + '.jpg')
                                    os.makedirs(os.path.dirname(save_path), exist_ok=True)
                                    cv2.imwrite(save_path, concatenated_image)

# Example usage:
root_directory = 'cropped'  # Directory containing cropped images
save_directory = 'concatenated_images'  # Directory to save concatenated images
save_concatenated_images(root_directory, save_directory)